In [1]:
# +-------------------------------------------------------------------------------------------------
# Author: CFolkers
# Ministry, Division, Branch: WLRS, GeoBC, Geospatial Services 
# Created Date: 06/04/24
# Updated Date: 
# Description: check overlaps from local file to those from a bcgw layer
# +-------------------------------------------------------------------------------------------------

In [29]:
# Modules
# +-------------------------------------------------------------------------------------------------
import os
import geopandas
import pandas
import oracledb
from sqlalchemy import create_engine
from getpass import getpass
from shapely import wkb
import openpyxl


In [30]:
# BCGW
# +-------------------------------------------------------------------------------------------------
user_nm= input("Enter BCGW user name: ")
bcgw_pass= getpass(prompt="Enter BCGW password: ")
host_nm= input("Enter BCGW host name: ")
service_nm= input ("enter BCGW Service name: ")

conn=oracledb.connect(user=user_nm, password= bcgw_pass, host=host_nm, port=1521, service_name=service_nm)
engine= create_engine('oracle+oracledb://', creator=lambda: conn)

In [31]:
# get layers function
# +-------------------------------------------------------------------------------------------------
def get_bcgw_layer(sql_query, oracle_conn, geo):
    geo_df=pandas.read_sql_query(sql_query, oracle_conn)
    print('got query from oracle')
    gdf= geopandas.read_postgis(sql_query,oracle_conn, geo )
    print('gdf returned')
    return gdf

In [32]:
# get pip layer
# +-------------------------------------------------------------------------------------------------
query= """SELECT sdo_util.to_wkbgeometry(SHAPE) AS wkb_geometry, t.* FROM WHSE_ADMIN_BOUNDARIES.PIP_CONSULTATION_AREAS_SP t """
pip=get_bcgw_layer(query, engine, 'wkb_geometry')
#set CRS
pip = pip.set_crs(3005, allow_override=True)
# print(pip.head())

got query from oracle
gdf returned


In [40]:
# get aquatic layer
# +-------------------------------------------------------------------------------------------------

local_shape="Protected_Areas_18Jan2024.shp"

if os.path.exists(r"/Ingenika_Chuyaza_Corridor shapes"):
    print('path exists')
    aquatic_hab=geopandas.read_file(local_shape)
    # print(aquatic_hab.head())
else:
    print('you dont see me')


path exists


In [ ]:
# intersect hab and PIP and group by
# +-------------------------------------------------------------------------------------------------
habitat_intersect= geopandas.sjoin(aquatic_hab, pip, how='inner')
# habitat_intersect= geopandas.overlay(aquatic_hab, pip,how='intersection')
habitat_PIP = habitat_intersect.groupby('Name')['cnsltn_area_name'].apply(list).reset_index()
print(habitat_PIP)

In [ ]:
max_length = max(habitat_PIP['cnsltn_area_name'].apply(len))
print(max_length)

In [44]:
for i in range(max_length):
    habitat_PIP[f'cnsltn_area_name_{i+1}'] = habitat_PIP['cnsltn_area_name'].apply(lambda x: x[i] if i < len(x) else None)

In [45]:
habitat_PIP.drop('cnsltn_area_name', axis=1, inplace=True)

In [46]:
# to excel
# +-------------------------------------------------------------------------------------------------
out_path=r"/Deliverables/Wədzih Yinè’_Overlaps.xlsx"
habitat_PIP.to_excel(out_path)